In [ ]:
!pip3 install "tqdm"

In [ ]:
import sys
import copy
import torch
import torch.nn as nn
from torch.backends import cudnn
from torch.autograd import Variable
import datetime
from sklearn.preprocessing import normalize
from pytz import timezone
import os
import math
import time
from torchvision.datasets import CIFAR100
from torchvision.transforms import transforms
import torchvision.utils
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sn
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
if not os.path.isdir('./OpenWorld'):
  !git clone https://github.com/Alessia-Sc/Open-World-Recognition.git OpenWorld

from OpenWorld.CIFAR100 import DatasetCifar100
from OpenWorld.ResNet import resnet32
from tqdm.notebook import tqdm

In [ ]:
LR=2.0
GAMMA=0.2
WEIGHT_DECAY=1e-5
MILESTONE=[49,63] #da provare a con 64

BATCH_SIZE=128
NUM_EPOCHS = 70
TOTAL_CLASSES = 100
NUM_CLASSES = 10
MOMENTUM=0.9
K=2000
DEVICE="cuda"

In [ ]:
class iCaRLNet(): 
    def __init__(self, feature_size, n_classes, lr=LR, momentum=MOMENTUM, gamma=GAMMA, weight_decay=WEIGHT_DECAY, milestone=MILESTONE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS):
        # Network architecture
        super(iCaRLNet, self).__init__()
        self.net = resnet32(num_classes=n_classes)
        self.feature_extractor = self.net.get_feat_ext
        self.forward = self.net.forward
        self.lr = lr
        self.gamma = gamma
        self.weight_decay = weight_decay
        self.milestone = milestone
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.n_classes = 0
        self.n_known = 0
        self.feature_size=feature_size
        self.momentum=momentum
        self.compute_SVM = True
        self.compute_forest = True
        self.compute_knn = True
        self.SVM=None
        self.forest = None
        self.KNN = None
       
        self.loss = nn.BCEWithLogitsLoss()

        self.compute_means = True
        self.compute_feat = True
        self.exemplar_means = []
        self.exemplar_sets = []
        self.exemplar_feat = []
        self.exemplar_lab = []
        
    def classify(self, x, cif): 
        
        if self.compute_means:
            exemplar_means = []
            #EXEMPLARS
            for P_y in self.exemplar_sets:  
                features = np.zeros((0,self.feature_size)) 
                sub = Subset(cif, P_y)
                dl = DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 
                with torch.no_grad():
                    for ind, ex, lab in dl:
                        ex = Variable(ex).cuda()
                        feature = self.feature_extractor(ex).data.cpu().numpy()
                        feature = normalize(feature, axis=1, norm='l2') 
                        features = np.concatenate((features,feature), axis=0) 

                features = torch.tensor(features)
                mu_y = features.mean(0).squeeze() 
                mu_y.data = mu_y.data / torch.norm(mu_y, p=2)  # L2 Normalize
                exemplar_means.append(mu_y)
                
            self.exemplar_means = exemplar_means #Tutte le medie di tutte le classi
            self.compute_means = False

        exemplar_means = self.exemplar_means
        means = torch.stack(exemplar_means)  
        means = torch.stack([means] * self.batch_size)
        means = means.transpose(1, 2)  

        feature = self.feature_extractor(x)  
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        feature = feature.unsqueeze(2) 
        feature = feature.expand_as(means)  
        feature = feature.cuda()
        means = means.cuda()

        dists = torch.sqrt((feature - means).pow(2).sum(1)).squeeze()  
        _, preds = dists.min(1)
        return preds

    def classify_SVM(self, x):  
      if self.compute_SVM:
        #TRAIN
        x_train = self.exemplar_feat
        y_train = self.exemplar_lab
        rbf=SVC(kernel='rbf', C=1)
        rbf.fit(x_train, y_train)
        self.SVM = rbf
        self.compute_SVM = False 

      feature = self.feature_extractor(x)  # (batch_size, feature_size)
      for i in range(feature.size(0)):  # Normalize
        feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
      x_test = feature.detach().cpu().numpy()

      y_pred=self.SVM.predict(x_test)
      y_pred=torch.Tensor(y_pred)
        
      return y_pred


    def classify_forest(self, x):  
      if self.compute_forest:
        x_train = self.exemplar_feat
        y_train = self.exemplar_lab
        forest=RandomForestClassifier(n_estimators=100)
        forest.fit(x_train, y_train)
        self.forest = forest
        self.compute_forest = False 
        
      feature = self.feature_extractor(x)  # (batch_size, feature_size)
      for i in range(feature.size(0)):  # Normalize
        feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
      x_test = feature.detach().cpu().numpy()

      y_pred=self.forest.predict(x_test)
      y_pred=torch.Tensor(y_pred)
        
      return y_pred

    def classify_KNN(self, x): 
        if self.compute_knn:
            x_train = self.exemplar_feat
            y_train = self.exemplar_lab
            neigh = KNeighborsClassifier(n_neighbors=10)
           
            neigh.fit(x_train, y_train)
            self.KNN=neigh
            self.compute_knn = False

        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
          feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        x_test=feature.detach().cpu().numpy()
            
        y_pred=self.KNN.predict(x_test)
        y_pred=torch.Tensor(y_pred)
        
        return y_pred



    def compute_exemplar_features(self, cif):
      if self.compute_feat:        
        exemplar_feat = np.zeros((0,self.feature_size))
        labs=[]
        for exemplar_list in self.exemplar_sets:  
            features = np.zeros((0,self.feature_size))
            sub = Subset(cif, exemplar_list)
            dl = DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 

            with torch.no_grad():
                for ind, ex, lab in dl:       # Extract feature for each exemplar in exemplar_list
                    ex = Variable(ex).cuda()
                    lab = torch.tensor([torch.tensor(cif.dict_class_label[c.item()]) for c in lab])
                    feature = self.feature_extractor(ex).data.cpu().numpy()
                    feature = normalize(feature, axis=1, norm='l2')
                    features = np.concatenate((features,feature), axis=0)
                    labs.extend(lab)
                    
            exemplar_feat=np.concatenate((exemplar_feat,features), axis=0)

        self.exemplar_lab=labs
        self.exemplar_feat = exemplar_feat
        self.compute_feat = False



    def construct_exemplar_set(self, images, m):
        features = np.zeros((0,self.feature_size))
        indices = np.zeros((0), dtype=int)
        dl = torch.utils.data.DataLoader(images, batch_size=self.batch_size,shuffle=False, num_workers=4)
        with torch.no_grad():
          for ind, img, lab in dl:
            x = Variable(img).cuda()
            feature = self.feature_extractor(x).data.cpu().numpy()
            feature = normalize(feature, axis=1, norm='l2')   
            features = np.concatenate((features,feature), axis=0)  
            indices = np.concatenate((indices,ind), axis=0)

        class_mean = np.mean(features, axis=0)
        class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

        exemplar_set = []
        exemplar_features = np.zeros((0,64))

        for k in range(1, int(m)+1):
            S = np.sum(exemplar_features, axis=0)
            phi = features    
            mu = class_mean 
            mu_p = 1.0 / k * (phi + S)
            mu_p = normalize(mu_p, axis=1, norm='l2')
            i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
            exemplar_set.append(indices[i])   
            addfeature =  np.expand_dims(features[i], axis=0)
            exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

            features = np.delete(features, i, 0)
            indices = np.delete(indices, i, 0)
            
        self.exemplar_sets.append(exemplar_set) 
        
    def reduce_exemplar_sets(self, m):
        for y, P_y in enumerate(self.exemplar_sets):
            self.exemplar_sets[y] = P_y[:int(m)]


    def exemplarIndexes(self):
        Indexes = []
        for P_y in self.exemplar_sets:
            Indexes += P_y
        return Indexes


    def update_representation(self, cifar, batchindexes):

        prev_model = copy.deepcopy(self)
        prev_model = prev_model.net.eval().cuda()
        self.net = self.net.cuda()
        self.compute_means = True
        self.compute_feat = True
        self.compute_SVM = True
        self.compute_forest = True
        self.compute_knn = True

        self.n_classes += 10

        # Form combined training set
        newindexes = []
        if self.n_classes > 10:    
            newindexes = self.exemplarIndexes() 
        newindexes += list(batchindexes)
        
        reprdata = Subset(cifar, newindexes)

        loader = DataLoader(reprdata, batch_size=self.batch_size,shuffle=True, num_workers=4, drop_last=True)

        optimizer = optim.SGD(self.net.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestone, gamma=self.gamma)

        for epoch in tqdm(range(self.num_epochs)):
            losses = []
            for _, images, labels in loader:  
                labels = torch.tensor([torch.tensor(train_ds.dict_class_label[c.item()]) for c in labels])
                images = Variable(torch.FloatTensor(images)).cuda()
                labels = Variable(labels).cuda()
                optimizer.zero_grad()
                g = self.forward(images)
                
                y_hot = F.one_hot(labels, self.n_classes).float().cuda()
                
                if self.n_known > 0:
                    q = prev_model.forward(images)
                    q = torch.sigmoid(q)
                    target = torch.cat((q[:,:self.n_known], y_hot[:,self.n_known:self.n_classes]), dim=1)
                    
                    loss = self.loss(g[:,:self.n_classes], target)  #Classification
                else:
                    loss = self.loss(g[:,:self.n_classes], y_hot[:,:self.n_classes])

                losses.append(loss.item())
                loss.backward()
                optimizer.step()
            
            scheduler.step()    
        

In [ ]:
def compute_acc(dl, icarl, cif):
    total = 0.0
    correct = 0.0
    correct_svm = 0.0
    correct_forest = 0.0
    correct_knn = 0.0

    for _, images, labels in dl:
        labels = torch.tensor([torch.tensor(cif.dict_class_label[c.item()]) for c in labels])
        images = Variable(images).cuda()
        preds = icarl.classify(images, cif)
       
        preds_svm = icarl.classify_SVM(images)        
        preds_forest = icarl.classify_forest(images)
        preds_knn = icarl.classify_KNN(images)

        total = total + len(labels)
        correct += (preds.data.cpu() == labels).sum()
        correct_knn += (preds_knn.data.cpu() == labels).sum()
        correct_svm += (preds_svm.data.cpu() == labels).sum()
        correct_forest += (preds_forest.data.cpu() == labels).sum()

    acc = 100 * correct / total
    acc_knn = 100 * correct_knn / total
    acc_svm = 100 * correct_svm / total
    acc_forest = 100 * correct_forest / total

    return acc, acc_knn, acc_svm, acc_forest

In [ ]:
train_ds = DatasetCifar100(split = "train", rand_seed=None) 
test_ds = DatasetCifar100(split = 'test', rand_seed=None)
randomlist = train_ds.classes
print("\nClasses in each group of 10:")
for i in range(10):
  print(f"\tGROUP {i+1}°: {randomlist[i:i+10]}")

torch.cuda.current_device()
torch.cuda._initialized = True

icarl = iCaRLNet(64, TOTAL_CLASSES)
icarl.net = icarl.net.to(DEVICE) #metto la rete sulla gpu

Files already downloaded and verified
Files already downloaded and verified

Classes in each group of 10:
	GROUP 1°: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52]
	GROUP 2°: [61, 49, 58, 92, 90, 68, 32, 28, 52, 47]
	GROUP 3°: [49, 58, 92, 90, 68, 32, 28, 52, 47, 87]
	GROUP 4°: [58, 92, 90, 68, 32, 28, 52, 47, 87, 1]
	GROUP 5°: [92, 90, 68, 32, 28, 52, 47, 87, 1, 41]
	GROUP 6°: [90, 68, 32, 28, 52, 47, 87, 1, 41, 93]
	GROUP 7°: [68, 32, 28, 52, 47, 87, 1, 41, 93, 6]
	GROUP 8°: [32, 28, 52, 47, 87, 1, 41, 93, 6, 88]
	GROUP 9°: [28, 52, 47, 87, 1, 41, 93, 6, 88, 12]
	GROUP 10°: [52, 47, 87, 1, 41, 93, 6, 88, 12, 38]


In [ ]:
vSVM=[]
vRF=[]
vKNN=[]
 
for s in range(10):

    print("\n")
    print('-' * 80)
    print(f"ITERATION: {(s+1)*10}/100")

    print("Loading training examples for classes", randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
    batchindexes = train_ds.get_indexes_from_labels(randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
    batch = Subset(train_ds, batchindexes)

    testindexes = test_ds.get_indexes_from_labels(randomlist[0:s*NUM_CLASSES + NUM_CLASSES])
    test_set = Subset(test_ds, testindexes)

    l=0
    for el in icarl.exemplar_sets:
      l+=len(el)  
    print("Batch size train: {} - Batch size test: {}".format(len(batch), len(test_set)))

    train_loader = DataLoader(batch, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True)
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True) #FALSE

    
    icarl.net.train()
    icarl.update_representation(train_ds, batchindexes)
    icarl.net.eval()
    m = K / icarl.n_classes 

    icarl.reduce_exemplar_sets(m)

    # Construct exemplar sets for new classes
    for y in randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES]:
        imagesInd = train_ds.get_indexes_from_labels([y]) 
        images = Subset(train_ds, imagesInd)
        icarl.construct_exemplar_set(images, m) 

    icarl.n_known = icarl.n_classes
    icarl.compute_exemplar_features(train_ds)

    acc, acc_knn, acc_svm, acc_forest= compute_acc(train_loader, icarl, train_ds)
    #print('Train Accuracy: %.2f' % acc)
    print('TRAIN Accuracy: {:.2f}, KNN Accuracy {:.2f}, SVM Accuracy: {:.2f}, Forest Accurracy: {:.2f}'.format(acc, acc_knn, acc_svm, acc_forest).rstrip())
        
    acc, acc_knn, acc_svm, acc_forest= compute_acc(test_loader, icarl, train_ds)
    #print('Test Accuracy: %.2f' % acc)
    print('TEST Accuracy: {:.2f}, KNN Accuracy {:.2f}, SVM Accuracy: {:.2f}, Forest Accurracy: {:.2f}'.format(acc, acc_knn, acc_svm, acc_forest).rstrip())
    
    vSVM.append(acc_svm.item())
    vRF.append(acc_forest.item())
    vKNN.append(acc_knn.item())

print("SVM")
print(vSVM)
print("RF")
print(vRF)
print("KNN")
print(vKNN)



--------------------------------------------------------------------------------
ITERATION: 10/100
Loading training examples for classes [36, 61, 49, 58, 92, 90, 68, 32, 28, 52]
Batch size train: 5000 - Batch size test: 1000



TRAIN Accuracy: 97.78, KNN Accuracy 97.72, SVM Accuracy: 97.94, Forest Accurracy: 97.48
TEST Accuracy: 87.39, KNN Accuracy 87.28, SVM Accuracy: 87.50, Forest Accurracy: 87.72


--------------------------------------------------------------------------------
ITERATION: 20/100
Loading training examples for classes [47, 87, 1, 41, 93, 6, 88, 12, 38, 91]
Batch size train: 5000 - Batch size test: 2000



TRAIN Accuracy: 91.37, KNN Accuracy 94.01, SVM Accuracy: 95.05, Forest Accurracy: 92.43
TEST Accuracy: 77.71, KNN Accuracy 77.55, SVM Accuracy: 77.81, Forest Accurracy: 76.51


--------------------------------------------------------------------------------
ITERATION: 30/100
Loading training examples for classes [81, 33, 8, 48, 60, 27, 50, 17, 56, 97]
Batch size train: 5000 - Batch size test: 3000



TRAIN Accuracy: 90.16, KNN Accuracy 90.02, SVM Accuracy: 94.11, Forest Accurracy: 88.40
TEST Accuracy: 71.74, KNN Accuracy 71.50, SVM Accuracy: 72.25, Forest Accurracy: 68.89


--------------------------------------------------------------------------------
ITERATION: 40/100
Loading training examples for classes [34, 42, 84, 66, 62, 26, 29, 51, 3, 72]
Batch size train: 5000 - Batch size test: 4000



TRAIN Accuracy: 86.36, KNN Accuracy 84.58, SVM Accuracy: 91.65, Forest Accurracy: 84.07
TEST Accuracy: 65.12, KNN Accuracy 64.74, SVM Accuracy: 64.99, Forest Accurracy: 61.39


--------------------------------------------------------------------------------
ITERATION: 50/100
Loading training examples for classes [39, 9, 37, 85, 13, 25, 11, 67, 99, 74]
Batch size train: 5000 - Batch size test: 5000


...


TRAIN Accuracy: 84.90, KNN Accuracy 84.17, SVM Accuracy: 89.12, Forest Accurracy: 83.91
TEST Accuracy: 55.94, KNN Accuracy 53.36, SVM Accuracy: 55.62, Forest Accurracy: 50.54


--------------------------------------------------------------------------------
ITERATION: 70/100
Loading training examples for classes [43, 73, 40, 55, 7, 78, 14, 10, 70, 44]
Batch size train: 5000 - Batch size test: 7000



TRAIN Accuracy: 78.93, KNN Accuracy 70.83, SVM Accuracy: 84.72, Forest Accurracy: 73.46
TEST Accuracy: 49.78, KNN Accuracy 48.47, SVM Accuracy: 49.62, Forest Accurracy: 45.92


--------------------------------------------------------------------------------
ITERATION: 80/100
Loading training examples for classes [0, 86, 79, 57, 75, 46, 83, 82, 22, 4]
Batch size train: 5000 - Batch size test: 8000



TRAIN Accuracy: 79.79, KNN Accuracy 75.12, SVM Accuracy: 86.48, Forest Accurracy: 80.57
TEST Accuracy: 48.01, KNN Accuracy 46.24, SVM Accuracy: 48.41, Forest Accurracy: 44.18


--------------------------------------------------------------------------------
ITERATION: 90/100
Loading training examples for classes [45, 18, 89, 5, 59, 21, 95, 96, 69, 16]
Batch size train: 5000 - Batch size test: 9000



TRAIN Accuracy: 77.52, KNN Accuracy 71.49, SVM Accuracy: 83.75, Forest Accurracy: 74.40
TEST Accuracy: 46.14, KNN Accuracy 43.75, SVM Accuracy: 46.34, Forest Accurracy: 41.23


--------------------------------------------------------------------------------
ITERATION: 100/100
Loading training examples for classes [98, 23, 80, 65, 76, 77, 20, 24, 94, 53]
Batch size train: 5000 - Batch size test: 10000



TRAIN Accuracy: 80.53, KNN Accuracy 76.52, SVM Accuracy: 85.10, Forest Accurracy: 79.55
TEST Accuracy: 44.55, KNN Accuracy 42.66, SVM Accuracy: 45.03, Forest Accurracy: 39.26
SVM
[87.5, 77.8125, 72.24864196777344, 64.99495697021484, 60.096153259277344, 55.621604919433594, 49.623844146728516, 48.41230010986328, 46.33928680419922, 45.03205108642578]
RF
[87.72321319580078, 76.51041412353516, 68.88587188720703, 61.39112854003906, 55.949520111083984, 50.543479919433594, 45.92013931274414, 44.178428649902344, 41.22768020629883, 39.262821197509766]
KNN
[87.27678680419922, 77.55208587646484, 71.50135803222656, 64.7429428100586, 58.95432662963867, 53.362770080566406, 48.466434478759766, 46.24496078491211, 43.75, 42.65825271606445]
